In [39]:
import pandas as pd
import numpy as np
import plotly.express as px
from src.catching import attempt_catch
from src.pokemon import PokemonFactory, StatusEffect

In [40]:
factory = PokemonFactory("pokemon.json")
df = pd.read_json("pokemon.json")
pokemons = list(df.columns)
pokeballs = ["pokeball", "ultraball", "fastball", "heavyball"]

In [41]:
def estimate_catchrate(pokemon_instance, pokeball, noise, n):
    return np.average([attempt_catch(pokemon_instance, pokeball, noise)[0] for _ in range(n)])

### a) Ejecutando la función 100 veces, para cada Pokemon en condiciones ideales (HP:100 %, LVL 100) ¿Cuál es la probabilidad de captura promedio para cada pokebola?

In [42]:
data = {}
for pokemon in pokemons:
    bicho = factory.create(pokemon, 100, StatusEffect.NONE, 1)
    data[pokemon] = {}
    for ball in pokeballs:
        data[pokemon][ball] = estimate_catchrate(bicho, ball, 0, 100)
print(data)

{'jolteon': {'pokeball': 0.06066666666666667, 'ultraball': 0.10433333333333333, 'fastball': 0.23733333333333334, 'heavyball': 0.035666666666666666}, 'caterpie': {'pokeball': 0.31933333333333336, 'ultraball': 0.6713333333333333, 'fastball': 0.337, 'heavyball': 0.306}, 'snorlax': {'pokeball': 0.034333333333333334, 'ultraball': 0.059333333333333335, 'fastball': 0.027, 'heavyball': 0.07633333333333334}, 'onix': {'pokeball': 0.055, 'ultraball': 0.12566666666666668, 'fastball': 0.05466666666666667, 'heavyball': 0.07833333333333334}, 'mewtwo': {'pokeball': 0.0023333333333333335, 'ultraball': 0.009, 'fastball': 0.017, 'heavyball': 0.0006666666666666666}}


In [43]:
average_rates = {
    ball: np.average([values[ball] for values in data.values()]) for ball in pokeballs
}
errors = {
    ball: np.std([values[ball] for values in data.values()]) for ball in pokeballs
}

df = pd.DataFrame({'Pokeball': pokeballs,
                   'Average Rate': list(average_rates.values()),
                   'Error': list(errors.values())})

print(df)

    Pokeball  Average Rate     Error
0   pokeball      0.094333  0.114341
1  ultraball      0.193933  0.242040
2   fastball      0.134600  0.129087
3  heavyball      0.099400  0.107210


In [44]:
fig = px.bar(
    data_frame=df,
    x="Pokeball", 
    y="Average Rate", 
    title="Average capture probability by Pokeball",
    labels={
        "x": "Pokeball type",
        "y": "Average capture rate"
    },
    error_y="Error"
)
fig.show()

Está clara la diferencia de efectividad entre pokebolas. Por ejemplo, la Ultraball tiene cerca del doble de efectividad que la Pokebola común. Sin embargo se pueden apreciar diferencias significativas en el error. Esto se debe a que no se segregan los datos por Pokemon, por lo cual la diferencia entre ellos se ve marcada en el error.

### 1b) ¿Es cierto que algunas pokebolas son más o menos efectivas dependiendo de propiedades intrinsecas de cada Pokemon? Justificar.

In [ ]:
data = {}
for pokemon in pokemons:
    bicho = factory.create(pokemon, 100, StatusEffect.NONE, 1)
    data[pokemon] = {}
    for ball in pokeballs:
        data[pokemon][ball] = estimate_catchrate(bicho, ball, 0, 3000)
print(data)

Volvemos a intentar las capturas, pero con un número de intentos una orden de magnitud mayor. Esto lo hacemos ya que con solo 100 intentos, había pokemons que no eran atrapados, lo cual dificultaba la comparación.

In [45]:
rates_by_pokemon = {
    pokemon: {
        pokeball: data[pokemon][pokeball] / data[pokemon]["pokeball"] for pokeball in data[pokemon].keys() if pokeball != "pokeball"
    } for pokemon in data.keys()
}
print(rates_by_pokemon)

{'jolteon': {'ultraball': 1.7197802197802197, 'fastball': 3.912087912087912, 'heavyball': 0.5879120879120879}, 'caterpie': {'ultraball': 2.1022964509394573, 'fastball': 1.0553235908141962, 'heavyball': 0.9582463465553235}, 'snorlax': {'ultraball': 1.7281553398058254, 'fastball': 0.7864077669902912, 'heavyball': 2.2233009708737863}, 'onix': {'ultraball': 2.284848484848485, 'fastball': 0.993939393939394, 'heavyball': 1.4242424242424243}, 'mewtwo': {'ultraball': 3.8571428571428563, 'fastball': 7.285714285714286, 'heavyball': 0.2857142857142857}}


In [46]:
table = []
for pokemon in rates_by_pokemon.keys():
    for pokeball in rates_by_pokemon[pokemon].keys():
        table.append([pokemon, pokeball, rates_by_pokemon[pokemon][pokeball]])
df = pd.DataFrame(table, columns=["pokemon", "pokeball", "rate"])
print(df)

     pokemon   pokeball      rate
0    jolteon  ultraball  1.719780
1    jolteon   fastball  3.912088
2    jolteon  heavyball  0.587912
3   caterpie  ultraball  2.102296
4   caterpie   fastball  1.055324
5   caterpie  heavyball  0.958246
6    snorlax  ultraball  1.728155
7    snorlax   fastball  0.786408
8    snorlax  heavyball  2.223301
9       onix  ultraball  2.284848
10      onix   fastball  0.993939
11      onix  heavyball  1.424242
12    mewtwo  ultraball  3.857143
13    mewtwo   fastball  7.285714
14    mewtwo  heavyball  0.285714


In [47]:
fig = px.bar(
    data_frame=df,
    x="pokemon", 
    y="rate", 
    title="Effectiveness of Special Pokeballs by Pokemon",
    labels={
        "pokemon": "Pokemon",
        "rate": "Effectiveness relative to a regular pokeball"
    },
    barmode="group",
    color="pokeball"
)
fig.show()

Podemos ver desde el grafico que es cierto que algunos tipos de pokebolas son (en comparacion a la pokebola base) más efectivas en ciertos Pokemones. Por ejemplo, las fastball son mucho mejores contra pokemons como Jolteon y Mewtwo, y las heavyballs son mejores contra Snorlax, pero peores contra Jolteon o Mewtwo.
En cambio, la ultraball no parece depender del Pokemon a atrapar. 

### 2a)¿Las condiciones de salud tienen algún efecto sobre la efectividad de la captura? Si es ası́, ¿Cuál es más o menos efectiva?

In [48]:
pokemon_test = "jolteon"
variants = { 
    status: factory.create(pokemon_test, 100, status, 1) for status in StatusEffect 
}
catchrate_by_variant = { 
    status: estimate_catchrate(pokemon, pokeballs[0], 1, 3000) for (status, pokemon) in variants.items() 
}
print(catchrate_by_variant)

{<StatusEffect.POISON: ('poison', 1.5)>: 0.08766666666666667, <StatusEffect.BURN: ('burn', 1.5)>: 0.10066666666666667, <StatusEffect.PARALYSIS: ('paralysis', 1.5)>: 0.096, <StatusEffect.SLEEP: ('sleep', 2)>: 0.12366666666666666, <StatusEffect.FREEZE: ('freeze', 2)>: 0.12233333333333334, <StatusEffect.NONE: ('none', 1)>: 0.06966666666666667}


In [49]:
fig = px.bar(
    x = list(map(lambda status : status.name, catchrate_by_variant.keys())), 
    y = list(catchrate_by_variant.values()), 
    title = "Average catch rate by Status Effect",
    labels= {
        "x": "Status Effect",
        "y": "Catch Rate"
    }
)
fig.show()

Podemos ver que los más efectivos son SLEEP y FREEZE. Mientras que POISON, BURN y PARALYSIS no llegan a ser tan efectivos. Sin embargo, todos los status es mejor que no tener ningún estado afectandolo.

### 2b) ¿Cómo afectan los puntos de vida a la efectividad de la captura? Sugerencia: Elegir uno o dos Pokemones y manteniendo el resto de los parámetros constantes, calcular la probabilidad de captura para distintos HP %

In [50]:
def get_catchrate_with_error(pokemon_instance, pokeball, noise, n):
    values = [attempt_catch(pokemon_instance, pokeball, noise)[1] for _ in range(n)]
    return {
        "rate": np.average(values), 
        "error": np.std(values)
    }

In [51]:
import decimal

def drange(x, y, jump):
  while x < y + jump:
    yield float(x)
    x += decimal.Decimal(jump)

health_vars = { 
    percentage: factory.create(pokemon_test, 100, StatusEffect.NONE, percentage) for percentage in drange(0, 1, 0.05) 
}

catchrate_by_health = { 
    percentage: get_catchrate_with_error(pokemon, pokeballs[0], 0.15, 3000) for (percentage, pokemon) in health_vars.items() 
}
print(catchrate_by_health)

{0.0: {'rate': 0.1751497919367319, 'error': 0.025757783930522728}, 0.05: {'rate': 0.17065538400785155, 'error': 0.02611612940595676}, 0.1: {'rate': 0.1639802262682593, 'error': 0.023932855375817057}, 0.15000000000000002: {'rate': 0.15881682278781611, 'error': 0.023883947284437147}, 0.2: {'rate': 0.1529821961825308, 'error': 0.02269556848246151}, 0.25: {'rate': 0.14675959374692366, 'error': 0.021934036852379126}, 0.30000000000000004: {'rate': 0.14083330088565488, 'error': 0.020814257446469434}, 0.35000000000000003: {'rate': 0.1354299346563606, 'error': 0.02120126784939077}, 0.4: {'rate': 0.129291040079578, 'error': 0.019231320380750153}, 0.45: {'rate': 0.12442805743616103, 'error': 0.01838209901053914}, 0.5: {'rate': 0.11764781836085755, 'error': 0.01716268534689979}, 0.55: {'rate': 0.11146528782341594, 'error': 0.016881226395303997}, 0.6000000000000001: {'rate': 0.10608961353906253, 'error': 0.01582259796896094}, 0.65: {'rate': 0.10046355091305575, 'error': 0.014970822033712423}, 0.700

In [52]:
values = {
    "x": [],
    "y": [],
    "upper_y": [],
    "lower_y": []
}
for item in catchrate_by_health.items():
    values["x"].append(item[0])
    values["y"].append(item[1]["rate"])
    values["upper_y"].append(item[1]["rate"] + item[1]["error"])
    values["lower_y"].append(item[1]["rate"] - item[1]["error"])
print(values)

{'x': [0.0, 0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001, 1.0], 'y': [0.1751497919367319, 0.17065538400785155, 0.1639802262682593, 0.15881682278781611, 0.1529821961825308, 0.14675959374692366, 0.14083330088565488, 0.1354299346563606, 0.129291040079578, 0.12442805743616103, 0.11764781836085755, 0.11146528782341594, 0.10608961353906253, 0.10046355091305575, 0.09437531399505213, 0.0882919484718497, 0.08257033805493538, 0.0769701392902271, 0.07125932375615783, 0.06503960418077646, 0.05864509664578625], 'upper_y': [0.20090757586725463, 0.1967715134138083, 0.18791308164407636, 0.18270077007225327, 0.17567776466499233, 0.16869363059930279, 0.1616475583321243, 0.15663120250575138, 0.14852236046032816, 0.14281015644670017, 0.13481050370775735, 0.12834651421871995, 0.12191221150802348, 0.11543437294676817, 0.10886061620566884, 0.101335

In [59]:
import plotly.graph_objs as go

fig = go.Figure([
    go.Scatter(
        x=values["x"],
        y=values["y"],
        line=dict(color='rgb(0,100,80)'),
        mode='lines+markers',
        showlegend=False,
        name="average rate"
    ),
    go.Scatter(
        x=values["x"]+values["x"][::-1], # x, then x reversed
        y=values["upper_y"]+values["lower_y"][::-1], # upper, then lower reversed
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False,
    )
])
fig.update_layout(
    title = f"Capture rate of {pokemon_test} vs its Health Points",
    xaxis_title="HP",
    yaxis_title="Catch rate",
)
fig.update_yaxes(
    rangemode="tozero",
    tickformat=".1%",
)
fig.update_xaxes(
    tickformat=".0%"
)
fig.show()

Podemos ver que a medida que aumentamos la vida del pokemon, menor será la posibilidad de captura de manera prácticamente lineal.

### 2c) ¿Qué parámetros son los que más afectan la probabilidad de captura?

Para ver que parámetros afectan más a la probabilidad de captura, calculamos la varianza de los valores calculados variando el parámetro y manteniendo los otros constantes.
Los parametros son:
- La vida
- El Efecto de estado
- La pokebola

Tomamos un Jolteon como ejemplo.

In [54]:
data_status = []
data_hp = []
data_pb = []
for hp in range(10):
    data_hp.append(attempt_catch(
        factory.create(pokemon_test, 100, StatusEffect.NONE, hp*0.1),
        pokeballs[0]
    )[1])
for status in StatusEffect:
    data_status.append(attempt_catch(
        factory.create(pokemon_test, 100, status, 1),
        pokeballs[0]
    )[1])
for pb in pokeballs:
    data_pb.append(attempt_catch(
        factory.create(pokemon_test, 100, StatusEffect.NONE, 1),
        pb
    )[1])

data_var = [np.var(data_status), np.var(data_hp), np.var(data_pb)]
df = pd.DataFrame(data_var,index=['Status Effect','HP','Pokeball'],columns=['Variance'])
px.bar(df,x=df.index,y="Variance",title="Effect of changing parameters on catch effectiveness",labels={
    "index": "Parameter"
}).show()

Para Jolteon en particular, la mayor varianza se obtiene cambiando el tipo de pokebola que se utiliza. Debido a que la efectividad de una pokebola depende de los atributos del pokemon, esto puede cambiar.

In [55]:
# Funcion devuelve una tupla (varianza_por_tipo, max y minimo por tipo)
def calculate_all_variances(pokemon, level, iterations):
    health_vars = { percentage: factory.create(pokemon_test, level, StatusEffect.NONE, percentage) for percentage in drange(0, 1, 0.05) }
    catchrate_by_health = { percentage: estimate_catchrate(pokemon, pokeballs[0], 1, iterations) for (percentage, pokemon) in health_vars.items() }

    variants = { status: factory.create(pokemon, level, status, 1) for status in StatusEffect }
    catchrate_by_variant = { status.name: estimate_catchrate(pokemon, pokeballs[0], 1, iterations) for (status, pokemon) in variants.items() }

    healthy_pokemon = factory.create(pokemon, level, StatusEffect.NONE, 1)
    catchrate_by_pokeball = { pokeball: estimate_catchrate(healthy_pokemon, pokeball, 1, 3000) for pokeball in pokeballs }
    return {
        "health": np.var(list(catchrate_by_health.values())),
        "status": np.var(list(catchrate_by_variant.values())),
        "pokeball": np.var(list(catchrate_by_pokeball.values()))
    }

all_pokemons_variances = { pokemon: calculate_all_variances(pokemon, 100, 3000) for pokemon in pokemons }

df = pd.DataFrame.from_dict(all_pokemons_variances)
df = df.assign(average=df.mean(axis=1))
print(df)
fig5 = px.bar(df, barmode="group",title="Effect of changing parameters on catch effectiveness by pokemon",labels={
    "index": "Parameter",
    "value": "Variance",
    "variable": "Pokemon"
})
fig5.show()

           jolteon  caterpie   snorlax      onix    mewtwo   average
health    0.001687  0.001555  0.001745  0.001323  0.001519  0.001566
status    0.000548  0.006162  0.000189  0.000473  0.000001  0.001475
pokeball  0.006687  0.011411  0.000547  0.000644  0.000028  0.003864


El gráfico nos muestra cuáles son las varianzas por cada Pokemon. Podemos ver que caterpie es afectado mucho más por el estado o la pokebola, mientras que mewtwo no es prácticamente afectado por estos parámetros. Por otro lado, la incidencia de la vida del pokemon en la efectividad no parece depender del pokemon en cuestión.
Agarrando el promedio de todas las varianzas, parece que la vida y la pokebola afectan bastante en comparacion al estado y nivel.

### 2d) Teniendo en cuenta uno o dos pokemones distintos: ¿Qué combinación de condiciones (propiedades mutables) y pokebola conviene utilizar para capturarlos?

In [61]:
max_catchrate = {
    pokemon: { "rate" : 0 } for pokemon in pokemons
}
for status in StatusEffect:
    for pokeball in pokeballs:
        for pokemon in max_catchrate.keys():
            catchrate = attempt_catch(
                factory.create(pokemon, 100, status, 1),
                pokeball
            )[1]
            if max_catchrate[pokemon]["rate"] < catchrate:
                max_catchrate[pokemon] = {
                    "rate": catchrate,
                    "status": status.name,
                    "pokeball": pokeball
                }
print(pd.DataFrame.from_dict(max_catchrate))
            

           jolteon   caterpie    snorlax       onix    mewtwo
rate        0.4688          1     0.1693     0.2344    0.0313
status       SLEEP      SLEEP      SLEEP      SLEEP     SLEEP
pokeball  fastball  ultraball  heavyball  ultraball  fastball


Considerando que entre más bajo sean los HP de los Pokemon, mayor será la chance de captura y las condiciones no cambian su efectividad en base al nivel de salud, directamente utilizamos HP = 100%. Podemos apreciar que SLEEP es el estado que más efectivo en general, junto con FREEZE, sin importar qué Pokemon estamos analizando. Sin embargo, la Pokebola a utilizar sí tiene grandes incidencias dependiendo del Pokemon que estemos analizando. Por ejemplo, la Heavyball es significativamente mejor en Snorlax que en Jolteon, para el cual la mejor pokebola es la Fastball.
Por lo tanto podemos afirmar que la Pokebola depende del Pokemon, mientras que siempre conviene utilizar SLEEP o FREEZE. Independientemente de esto, siempre se debe buscar que el Pokemon tenga el menor porcentaje de salud posible.

<insert blabla>

### 2e) A partir del punto anterior, ¿serı́a efectiva otra combinación de parámetros teniendo en cuenta un nivel del pokemon más bajo (o más alto)?

Repetimos el anterior pero con un nivel de 40 en vez de 100

In [62]:
max_catchrate = {
    pokemon: { "rate" : 0 } for pokemon in pokemons
}
for status in StatusEffect:
    for pokeball in pokeballs:
        for pokemon in max_catchrate.keys():
            catchrate = attempt_catch(
                factory.create(pokemon, 40, status, 1),
                pokeball
            )[1]
            if max_catchrate[pokemon]["rate"] < catchrate:
                max_catchrate[pokemon] = {
                    "rate": catchrate,
                    "status": status.name,
                    "pokeball": pokeball
                }
print(pd.DataFrame.from_dict(max_catchrate))

           jolteon   caterpie    snorlax       onix    mewtwo
rate        0.4688          1     0.1693     0.2344    0.0313
status       SLEEP      SLEEP      SLEEP      SLEEP     SLEEP
pokeball  fastball  ultraball  heavyball  ultraball  fastball


El nivel no parecería afectar qué parametros usar.